In [7]:
import pandas as pd

In [8]:
pd.set_option('display.max_colwidth', None)

# Установка максимального количества отображаемых столбцов
pd.set_option('display.max_columns', None)

# Установка максимального количества отображаемых строк
pd.set_option('display.max_rows', None)

pd.set_option('display.max_info_columns', 200)

In [9]:
df_egrul = pd.read_csv("PSB/data/egrul_final.csv")
df_kad_arbitr = pd.read_csv("PSB/data/kad_arbitr_final.csv")
df_organizations = pd.read_csv("PSB/data/organizations_final.csv")
df_finances = pd.read_csv("PSB/data/finances_final.csv")
# df_egrul.head()

df_egrul = df_egrul.rename(columns={'data_ИНН': 'inn'})
df_egrul = df_egrul.drop(columns="Unnamed: 0", axis=1)
df_kad_arbitr = df_kad_arbitr.drop(columns="Unnamed: 0", axis=1)

df_egrul_kad_arbitr_merged = pd.merge(df_egrul, df_kad_arbitr, on='inn', how='outer')
df_egrul_kad_arbitr_merged["Недостоверный_адрес"] = df_egrul_kad_arbitr_merged["Недостоверный_адрес"].astype(int)
df_egrul_kad_arbitr_merged["data_Санкции"] = df_egrul_kad_arbitr_merged["data_Санкции"].astype(int)
df_egrul_kad_arbitr_merged["data_НелегалФин"] = df_egrul_kad_arbitr_merged["data_НелегалФин"].astype(int)
# df_egrul_kad_arbitr_merged["Статус"] = df_egrul_kad_arbitr_merged["Статус"].astype(int)
df_egrul_kad_arbitr_merged["Массовый_учредитель"] = df_egrul_kad_arbitr_merged["Массовый_учредитель"].astype(int)
df_egrul_kad_arbitr_merged["Есть_банкротство"] = df_egrul_kad_arbitr_merged["Есть_банкротство"].astype(int)
df_egrul_kad_arbitr_merged["Недобросовестный_поставщик"] = df_egrul_kad_arbitr_merged["Недобросовестный_поставщик"].astype(int)
# Массовый_учредитель
df_egrul_kad_arbitr_merged = df_egrul_kad_arbitr_merged.drop(columns=["data_ОКАТО_Наим", "data_ОКВЭД_Наим", "data_ОКОГУ_Наим", "data_НаимПолн"])
# df_egrul_kad_arbitr_merged.to_csv("PSB/df_egrul_kad_arbitr_merged_v020.csv")
# df_egrul_kad_arbitr_merged.head()

df_egrul_kad_arbitr_organizations_merged = pd.merge(df_egrul_kad_arbitr_merged, df_organizations, on='inn', how='outer')
df_egrul_kad_arbitr_organizations_merged = df_egrul_kad_arbitr_organizations_merged.drop(columns=["Unnamed: 0", "name", "Статус"])

ohe_data = pd.get_dummies(df_egrul_kad_arbitr_organizations_merged["Категория_МСП"], prefix='Категория_МСП').astype(int)
df_egrul_kad_arbitr_organizations_merged = pd.concat([df_egrul_kad_arbitr_organizations_merged, ohe_data], axis=1)
df_egrul_kad_arbitr_organizations_merged = df_egrul_kad_arbitr_organizations_merged.drop(columns=["Категория_МСП"])
df_egrul_kad_arbitr_organizations_merged["Количество_дел"] = df_egrul_kad_arbitr_organizations_merged["Количество_дел"].fillna(df_egrul_kad_arbitr_organizations_merged["Количество_дел"].mode()[0])
df_egrul_kad_arbitr_organizations_merged["Общая_сумма"] = df_egrul_kad_arbitr_organizations_merged["Общая_сумма"].fillna(df_egrul_kad_arbitr_organizations_merged["Общая_сумма"].mode()[0])
df_egrul_kad_arbitr_organizations_merged["Среднее_по_делу"] = df_egrul_kad_arbitr_organizations_merged["Среднее_по_делу"].fillna(df_egrul_kad_arbitr_organizations_merged["Среднее_по_делу"].mode()[0])
df_egrul_kad_arbitr_organizations_merged["Максимальная_сумма"] = df_egrul_kad_arbitr_organizations_merged["Максимальная_сумма"].fillna(df_egrul_kad_arbitr_organizations_merged["Максимальная_сумма"].mode()[0])
df_egrul_kad_arbitr_organizations_merged["Последнее_дело_дата"] = df_egrul_kad_arbitr_organizations_merged["Последнее_дело_дата"].fillna(df_egrul_kad_arbitr_organizations_merged["Последнее_дело_дата"].mode()[0])
df_egrul_kad_arbitr_organizations_merged["Первое_дело_дата"] = df_egrul_kad_arbitr_organizations_merged["Первое_дело_дата"].fillna(df_egrul_kad_arbitr_organizations_merged["Первое_дело_дата"].mode()[0])
df_egrul_kad_arbitr_organizations_merged["Количество_дел_в_производстве"] = df_egrul_kad_arbitr_organizations_merged["Количество_дел_в_производстве"].fillna(df_egrul_kad_arbitr_organizations_merged["Количество_дел_в_производстве"].mode()[0])
df_egrul_kad_arbitr_organizations_merged["Всего_дел_в_системе"] = df_egrul_kad_arbitr_organizations_merged["Всего_дел_в_системе"].fillna(df_egrul_kad_arbitr_organizations_merged["Всего_дел_в_системе"].mode()[0])
df_egrul_kad_arbitr_organizations_merged["Длительность_судебной_истории"] = df_egrul_kad_arbitr_organizations_merged["Длительность_судебной_истории"].fillna(df_egrul_kad_arbitr_organizations_merged["Длительность_судебной_истории"].mode()[0])
df_egrul_kad_arbitr_organizations_merged['data_ОКВЭД_Код'] = df_egrul_kad_arbitr_organizations_merged['data_ОКВЭД_Код'].apply(lambda x: int(x.split('.')[0]))
df_egrul_kad_arbitr_organizations_merged.head()
# df_egrul_kad_arbitr_organizations_merged.info()
df_egrul_kad_arbitr_organizations_merged.to_csv("PSB/df_egrul_kad_arbitr_organizations_merged_final.csv")

df_all_merged = pd.merge(df_egrul_kad_arbitr_organizations_merged, df_finances, on='inn', how='outer')
# df_all_merged = df_all_merged.drop(columns=["inn.1", "inn.2"], axis=1)
df_all_merged.to_csv("PSB/data/df_merged_pre_final.csv")
df_all_merged.info()
df_all_merged.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 49 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   inn                                15000 non-null  int64  
 1   data_ОКВЭД_Код                     15000 non-null  int64  
 2   data_Налоги_СумУпл                 15000 non-null  float64
 3   data_Налоги_СумНедоим              15000 non-null  float64
 4   data_УстКап_Сумма                  15000 non-null  float64
 5   data_Санкции                       15000 non-null  int64  
 6   data_НелегалФин                    15000 non-null  int64  
 7   Возраст_компании                   15000 non-null  int64  
 8   Недостоверный_адрес                15000 non-null  int64  
 9   Особый_налоговый_режим             15000 non-null  int64  
 10  Массовый_учредитель                15000 non-null  int64  
 11  Есть_банкротство                   15000 non-null  int

,inn,data_ОКВЭД_Код,data_Налоги_СумУпл,data_Налоги_СумНедоим,data_УстКап_Сумма,data_Санкции,data_НелегалФин,Возраст_компании,Недостоверный_адрес,Особый_налоговый_режим,Массовый_учредитель,Есть_банкротство,Недобросовестный_поставщик,Среднесписочная_численность,Количество_дел,Общая_сумма,Среднее_по_делу,Максимальная_сумма,Последнее_дело_дата,Первое_дело_дата,Количество_дел_в_производстве,Всего_дел_в_системе,Длительность_судебной_истории,status_Действующая организация,status_Деятельность прекращена,status_Процесс банкротства,status_Процесс исключения,status_Процесс реорганизации,status_Стадия ликвидации,Категория_МСП_МАЛОЕ ПРЕДПРИЯТИЕ,Категория_МСП_МИКРОПРЕДПРИЯТИЕ,Категория_МСП_Неизвестно,Категория_МСП_СРЕДНЕЕ ПРЕДПРИЯТИЕ,Unnamed: 0,2022_1300,2022_1600,2022_1700,2022_2110,2022_2400,2023_1300,2023_1600,2023_1700,2023_2110,2023_2400,2024_1300,2024_1600,2024_1700,2024_2110,2024_2400
0,1001001390,2,2.478546e+08,2.072310e+07,15583520.0,0,0,30,0,0,0,0,0,163,589.0,7.658237e+08,1.300210e+06,9.239343e+07,2025.0,2011.0,0.0,589.0,14.0,1,0,0,0,0,0,0,0,1,0,3,265575000.0,868710000.0,868710000.0,1.771477e+09,1441000.0,223839000.0,568964000.0,568964000.0,1.979607e+09,-100498000.0,236864000.0,724687000.0,724687000.0,2.225792e+09,900000.0
1,1001004095,20,1.695939e+07,0.000000e+00,32400.0,0,0,32,0,0,0,0,0,37,34.0,3.694155e+07,1.086516e+06,1.803287e+07,2024.0,2014.0,0.0,34.0,10.0,1,0,0,0,0,0,1,0,0,0,4,171441000.0,189731000.0,189731000.0,1.706430e+08,21910000.0,171235000.0,193352000.0,193352000.0,1.758600e+08,7243000.0,169797000.0,192447000.0,192447000.0,2.023990e+08,2183000.0
2,1001028755,46,4.668453e+06,0.000000e+00,10000.0,0,0,13,0,0,1,0,0,39,25.0,2.799794e+07,1.119918e+06,5.647302e+06,2023.0,2015.0,0.0,25.0,8.0,1,0,0,0,0,0,1,0,0,0,1,73295000.0,456052000.0,456052000.0,8.261840e+08,-161000.0,69080000.0,499412000.0,499412000.0,8.394260e+08,-4219000.0,69173000.0,532301000.0,532301000.0,9.074280e+08,93000.0
3,1001095007,46,5.535989e+06,0.000000e+00,210000.0,0,0,25,0,0,0,0,0,3,4.0,4.064346e+06,1.016087e+06,3.815452e+06,2023.0,2013.0,0.0,4.0,10.0,1,0,0,0,0,0,1,0,0,0,2,18625000.0,38058000.0,38058000.0,9.993000e+07,3376000.0,17473000.0,24702000.0,24702000.0,7.620400e+07,308000.0,16949000.0,21702000.0,21702000.0,9.670800e+07,1085000.0
4,1001098784,10,1.331751e+07,1.269584e+08,9850000.0,0,0,25,0,0,0,1,0,4,129.0,7.341908e+08,5.691401e+06,2.070000e+08,2024.0,2003.0,0.0,129.0,21.0,0,0,1,0,0,0,1,0,0,0,0,-238777000.0,139897000.0,139897000.0,1.852220e+08,-37066000.0,-273831000.0,123352000.0,123352000.0,1.342630e+08,-37231000.0,-296703000.0,103538000.0,103538000.0,6.583000e+06,-8904000.0
